**Initializing Spark**

In [53]:
from pyspark.sql import SparkSession

In [54]:
spark = SparkSession.builder.getOrCreate()

**Praject start Timestamp**

In [63]:
import datetime,time
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

2022-01-20 10:47:23


In [64]:
order_details_data = [(10248,11,14,12 ),
(10248,42,9,10  ),
(10248,72,34,5  ),
(10249,14,18,9  ),
(10249,51,42,40 ),
(10253,31,10,20 ),
(10253,39,14,42 ),
(10253,49,16,40 ),
(10274,71,17,20 ),
(10274,72,27,7  ),
(10275,24,3,12  ),
(10275,59,44,6  ),
(10296,11,16,12 ),
(10296,16,13,30 ),
(10296,69,28,15 ) ]


In [65]:
orders_data = [(10248,3,'1996-07-04'),
(10249,1,'1996-07-05'),
(10253,2,'1996-07-10'),
(10274,3,'1996-08-06'),
(10275,4,'1996-08-07'),
(10296,5,'1996-09-03')]

**Creating Order and order_details dataframe**

In [66]:
order_details_df = spark.createDataFrame(order_details_data,['orderid','productid','unitprice','quantity' ] )

In [67]:
orders_df = spark.createDataFrame(orders_data,['orderid','customerid','orderdate' ] )

**Creating views**

In [68]:
order_details_df.createOrReplaceTempView("order_details")
orders_df.createOrReplaceTempView("orders")

In [69]:
spark.sql("""

select year,month,customerid,total_monthly_order_value from (
select year,month,customerid,total_monthly_order_value,
row_number() over (partition by year,month order by total_monthly_order_value desc ) as rn

from (
select customerid,year,month,sum(order_value) as total_monthly_order_value
from 
(
select year(orderdate) as year,month(orderdate) as month,
unitprice*quantity as order_value,*
from orders inner join order_details 
on orders.orderid=order_details.orderid
)
group by 1,2,3 
)
) where rn=1 order by year,month asc
""").show()

+----+-----+----------+-------------------------+
|year|month|customerid|total_monthly_order_value|
+----+-----+----------+-------------------------+
|1996|    7|         1|                     1842|
|1996|    8|         3|                      529|
|1996|    9|         5|                     1002|
+----+-----+----------+-------------------------+



**Project End Timestamp**

In [70]:
import datetime
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

2022-01-20 10:47:36
